In [1]:
import geopandas 
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from shapely.geometry import Point, Polygon, LineString
import seaborn as sns 
sns.set()
import descartes
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians
import numpy as np

lakes = geopandas.read_file("Lakes_and_Rivers")
madison = geopandas.read_file("City_Limit")

In [2]:
#https://data-cityofmadison.opendata.arcgis.com/datasets/metro-transit-ridership-by-route-weekday?geometry=-89.671%2C42.985%2C-89.23%2C43.161
weekday_bus = geopandas.read_file("Metro_Transit_Ridership_by_Route_Weekday")

weekday_bus


,OBJECTID,StopID,Route,Lat,Lon,DailyBoard,DotSize,geometry
0,19341,7215,16,43.084090,-89.325022,9.72,97179.20,POINT (-89.32501598855713 43.08408205170329)
1,19342,7215,31,43.084090,-89.325022,4.01,40110.65,POINT (-89.32501598855713 43.08408205170329)
2,19343,7215,37,43.084090,-89.325022,2.52,25165.14,POINT (-89.32501598855713 43.08408205170329)
3,19344,7215,38,43.084090,-89.325022,10.26,102564.00,POINT (-89.32501598855713 43.08408205170329)
4,19345,7219,35,43.067971,-89.281743,0.33,3338.55,POINT (-89.28173700253664 43.06796305245022)
5,19346,7223,35,43.067560,-89.291297,0.17,1669.27,POINT (-89.29129100010601 43.0675520522667)
6,19347,7223,39,43.067560,-89.291297,3.34,33376.46,POINT (-89.29129100010601 43.0675520522667)
7,19348,7226,31,43.051751,-89.292043,0.17,1671.28,POINT (-89.29203700073032 43.05174305489181)
8,19349,7230,32,43.096102,-89.291095,0.54,5389.78,POINT (-89.2910889969265 43.09609404756924)
9,19350,7232,16,43.062188,-89.320363,58.48,584750.70,POINT (-89.32035699229068 43.06218005515254)


In [3]:
#metro data
metro_data = geopandas.read_file("MetroBusRoutes.gdb")
list_metro = []
for index, row in metro_data.iterrows():
    list_metro.append(Point(float(row["stop_lon"]),float(row["stop_lat"])))

metro_series = pd.Series(list_metro)
metro_coord = pd.DataFrame({"Coordinates": metro_series})
metro_coord = geopandas.GeoDataFrame(metro_coord, geometry='Coordinates')

#BRT data 
#https://data-cityofmadison.opendata.arcgis.com/datasets/bus-rapid-transit-stops
# using new data for brt from the madison data instead of the BRT.gdb
brt_data = geopandas.read_file("Bus_Rapid_Transit_Stops")
list_brt = []
for index, row in brt_data.iterrows():
    list_brt.append(Point(float(row["stop_lon"]),float(row["stop_lat"])))

brt_series = pd.Series(list_brt)
brt_coord = pd.DataFrame({"Coordinates": brt_series})
brt_coord = geopandas.GeoDataFrame(brt_coord, geometry='Coordinates')



In [4]:
#ref _ https://stackoverflow.com/questions/10202570/find-row-where-values-for-column-is-maximal-in-a-pandas-dataframe
# ref _ http://postgis.org/docs/ST_Distance.html
weekday_bus['brt_index'] = ''
weekday_bus['distance_to_brt'] = ''

for index, row in weekday_bus.iterrows():
    list_bus = []
    for index1, row1 in brt_data.iterrows(): 
        dist = row['geometry'].distance(row1['geometry'])
        list_bus.append(dist)
    list_bus_df = pd.DataFrame(list_bus)
    list_bus_df.columns =['distance_from_nearest_brt']

    index_of_brt = list_bus_df['distance_from_nearest_brt'].idxmin()
    distance_to_nearest_brt = list_bus_df['distance_from_nearest_brt'].min()

    weekday_bus.at[index, 'brt_index'] = index_of_brt
    weekday_bus.at[index, 'distance_to_brt'] = distance_to_nearest_brt

In [5]:
weekday_bus

,OBJECTID,StopID,Route,Lat,Lon,DailyBoard,DotSize,geometry,brt_index,distance_to_brt
0,19341,7215,16,43.084090,-89.325022,9.72,97179.20,POINT (-89.32501598855713 43.08408205170329),17,0.026139
1,19342,7215,31,43.084090,-89.325022,4.01,40110.65,POINT (-89.32501598855713 43.08408205170329),17,0.026139
2,19343,7215,37,43.084090,-89.325022,2.52,25165.14,POINT (-89.32501598855713 43.08408205170329),17,0.026139
3,19344,7215,38,43.084090,-89.325022,10.26,102564.00,POINT (-89.32501598855713 43.08408205170329),17,0.026139
4,19345,7219,35,43.067971,-89.281743,0.33,3338.55,POINT (-89.28173700253664 43.06796305245022),22,0.0637091
5,19346,7223,35,43.067560,-89.291297,0.17,1669.27,POINT (-89.29129100010601 43.0675520522667),20,0.0594231
6,19347,7223,39,43.067560,-89.291297,3.34,33376.46,POINT (-89.29129100010601 43.0675520522667),20,0.0594231
7,19348,7226,31,43.051751,-89.292043,0.17,1671.28,POINT (-89.29203700073032 43.05174305489181),18,0.0720935
8,19349,7230,32,43.096102,-89.291095,0.54,5389.78,POINT (-89.2910889969265 43.09609404756924),22,0.0342745
9,19350,7232,16,43.062188,-89.320363,58.48,584750.70,POINT (-89.32035699229068 43.06218005515254),17,0.0471696


In [6]:
# weekday_bus.to_csv('weekday_bus_with_brt_distance.gpkg', encoding='utf-8', index=False)

In [7]:
weekday_bus.to_file("weekday_bus_with_brt_distance.gpkg", driver="GPKG")

CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_weekday_bus_with_brt_distance_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona/_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE VIRTUAL TABLE "rtree_weekday_bus_with_brt_distance_geom" USING rtree(id, minx, maxx, miny, maxy)) failed: no such module: rtree'
